In [1]:
import os
from urllib.request import urlretrieve
import time
import zipfile
from tqdm import tqdm
import requests
import pandas as pd

In [2]:
year = 2016
quarter = 2
key_list = []
while True:
    year_str = str(year)[-2:]
    key_str = f'{quarter}T{year_str}'
    key_list.append(key_str)
    quarter += 1
    if key_str == '1T24':
        break
    if quarter == 5:
        quarter = 1
        year += 1
key_list

['2T16',
 '3T16',
 '4T16',
 '1T17',
 '2T17',
 '3T17',
 '4T17',
 '1T18',
 '2T18',
 '3T18',
 '4T18',
 '1T19',
 '2T19',
 '3T19',
 '4T19',
 '1T20',
 '2T20',
 '3T20',
 '4T20',
 '1T21',
 '2T21',
 '3T21',
 '4T21',
 '1T22',
 '2T22',
 '3T22',
 '4T22',
 '1T23',
 '2T23',
 '3T23',
 '4T23',
 '1T24']

In [3]:
value_list = []
for key in key_list:
    quarter = key[0]
    year = '20' + key[2:]
    url_1 = 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/'
    url_2 = f'eph/EPH_usu_{quarter}_Trim_{year}_txt.zip'
    value_list.append(url_1 + url_2)
value_list

['https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2016_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2016_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2016_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/ep

In [4]:
url_dict = dict(zip(key_list, value_list))
url_dict

{'2T16': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2016_txt.zip',
 '3T16': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2016_txt.zip',
 '4T16': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2016_txt.zip',
 '1T17': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1_Trim_2017_txt.zip',
 '2T17': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2017_txt.zip',
 '3T17': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2017_txt.zip',
 '4T17': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2017_txt.zip',
 '1T18': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1_Trim_2018_txt.zip',
 '2T18': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2018_txt.zip',
 '3T18': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2018_txt.zip',
 '4T18': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/

In [5]:
# Create the "datos" folder if it doesn't exist
if 'datos' not in os.listdir():
    os.makedirs('datos')

for key in tqdm(url_dict, desc='Downloading files'):
    url = url_dict[key]
    response = requests.get(url, stream=True)
    
    # If the website responds 
    if response.status_code == 200:
        with open(f'datos/{key}.zip', 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)

        # Try to avoid DDoS detection
        time.sleep(2)

    else:
        print(f'Failed to download {key}')

In [7]:
def check_zip_integrity(filepath):
    try:
        with zipfile.ZipFile(filepath, 'r') as z:
            bad_file = z.testzip()
            if bad_file is not None:
                print(f"{filepath} is corrupted.")
                return filepath

    except zipfile.BadZipFile:
        print(f"{filepath} is not a valid zip file.")
        return False
    
for file in os.listdir('datos'):
    check_zip_integrity(f'datos/{file}')

datos/4T16.zip is not a valid zip file.
datos/1T17.zip is not a valid zip file.
datos/3T16.zip is not a valid zip file.
datos/2T16.zip is not a valid zip file.


In [3]:
%%time
drop_list = ['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'H15', 'AGLOMERADO', 'MAS_500', 
              'CH03', 'CH05', 'CH09', 'CH11', 'CH12', 'CH13', 'CH14', 'CH15',
              'CH15_COD', 'CH16', 'CH16_COD', 'IMPUTA', 'PP02C1', 'PP02C2',
              'PP02C3', 'PP02C4', 'PP02C5', 'PP02C6', 'PP02C7', 'PP02C8', 
              'PP03C', 'PP04B2', 'PP04B3_MES', 'PP04B3_ANO', 
              'PP04B3_DIA', 'PP04C99', 'PP05C_2', 'PP05C_3', 
              'PP05E', 'PP05F', 'PP05H', 'PP06A', 'PP06C', 'PP06D', 'PP06E', 
              'PP06H', 'PP07A', 'PP07D', 'PP07E', 'PP07F1', 'PP07F2', 
              'PP07F3', 'PP07F4', 'PP07F5', 'PP07G1', 'PP07G2', 'PP07G3', 
              'PP07G4', 'PP07G_59', 'PP07J', 'PP08D1', 'PP08D4', 'PP08F1', 
              'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3', 'PP09A', 'PP09A_ESP',
              'PP09B', 'PP09C_ESP', 'DECOCUR', 'IDECOCUR', 'RDECOCUR', 
              'GDECOCUR', 'PDECOCUR', 'ADECOCUR', 'TOT_P12', 
              'DECINDR', 'IDECINDR', 'RDECINDR', 'GDECINDR', 'PDECINDR', 
              'ADECINDR', 'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 
              'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 
              'T_VI', 'ITF', 'DECIFR', 'DECIFR', 'RDECIFR', 'GDECIFR', 'PDECIFR',
              'ADECIFR', 'IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 
              'PDECCFR', 'ADECCFR', 'PONDIH', 'PP09C_ESP', 'PP10A', 'PP10C', 
              'PP10D', 'PP10E', 'PP11A', 'PP11B_COD', 'PP11B1', 'PP11B2_MES', 
              'PP11B2_ANO', 'PP11B2_DIA', 'PP11C', 'PP11C99', 'PP11C99', 
              'PP11D_COD', 'PP11G_ANO', 'PP11G_MES', 'PP11G_DIA', 'PP11L', 
              'PP11L1', 'PP11M', 'PP11N', 'PP11O', 'PP11P', 'PP11Q', 'PP11R', 
              'PP11S', 'PP11T'
            ]

data_dict = {}
bad_files = []

for i in enumerate(files_list):
    try:
        data = pd.read_excel(i[1])
        data = data.drop(columns=drop_list)
        data = data.loc[data['REGION'] == 43]
        data_dict[f'eph{i[0]}'] = data
    except KeyError:
        bad_files.append(i[1])

WARNING *** file size (98984336) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (96425626) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (99315329) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (97454840) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
CPU times: total: 21min 7s
Wall time: 21min 10s


In [4]:
bad_files

[]

In [5]:
eph = pd.concat([data_dict['eph1'], data_dict['eph2']], axis=0)
del(data_dict['eph1'])
del(data_dict['eph2'])

for i in enumerate(data_dict):
    eph =  pd.concat([eph, data_dict[i[1]]], axis=0)

In [6]:
eph

,ANO4,TRIMESTRE,REGION,PONDERA,CH04,CH06,CH07,CH08,CH10,NIVEL_ED,...,PP07C,PP07H,PP07I,PP07K,PP09C,P21,PONDIIO,P47T,PONDII,IDECIFR
0,2017,1,43,861,2,18,1,4.0,2,2,...,NaN,NaN,NaN,NaN,NaN,0,861,1000.0,1088,
1,2017,1,43,861,2,13,5,4.0,1,3,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
2,2017,1,43,861,1,15,5,4.0,1,3,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
3,2017,1,43,861,2,11,5,4.0,1,3,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
4,2017,1,43,861,1,7,5,4.0,1,1,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48540,2022,4,43,753,2,5,5,1.0,1,7,...,NaN,NaN,NaN,NaN,NaN,0,753,0.0,753,8.0
48541,2022,4,43,753,1,3,5,1.0,3,7,...,NaN,NaN,NaN,NaN,NaN,0,753,0.0,753,8.0
48542,2022,4,43,692,1,46,1,1.0,1,5,...,2.0,2.0,1.0,4.0,0.0,60000,933,60000.0,1089,9.0
48543,2022,4,43,692,2,44,1,1.0,2,6,...,2.0,1.0,0.0,1.0,0.0,160000,1033,160000.0,1092,9.0


In [11]:
eph.loc[eph['ESTADO'] == 1, :].shape

(183461, 44)

In [12]:
eph.loc[eph['P21'] != 0, :].shape

(179030, 44)

In [7]:
eph.to_csv('EPH.csv', index=False)